# Learning Rate Schedule 

* A CNN model is constructed to train on CIFAR-10

* The following learning rate decay schedules are used : constant learning rate, time-based decay, step decay, exponential decay
    
* Model performance of using different learning rate schedules are compared

In [2]:
from __future__ import print_function
import numpy as np
import math

import matplotlib.pyplot as plt
# If you're running this notebook locally, uncomment this line
# %matplotlib inline  

import keras
from keras import backend as K
from keras.datasets import cifar10
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD
from keras.models import model_from_json
from keras.callbacks import LearningRateScheduler

Using TensorFlow backend.

# Load CIFAR-10 data

In [4]:
batch_size = 64
num_classes = 2
epochs = 50

# input image dimensions
img_rows, img_cols = 32, 32   

# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = cifar10.load_data()   

# Only look at cats [=3] and dogs [=5]
train_picks = np.ravel(np.logical_or(y_train==3,y_train==5))  
test_picks = np.ravel(np.logical_or(y_test==3,y_test==5))     

y_train = np.array(y_train[train_picks]==5,dtype=int)
y_test = np.array(y_test[test_picks]==5,dtype=int)

X_train = X_train[train_picks]
X_test = X_test[test_picks]

# Re-format according to Keras's setting
# channel: RGB channels
if K.image_data_format() == 'channels_first':
    X_train = X_train.reshape(X_train.shape[0], 3, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 3, img_rows, img_cols)
    input_shape = (3, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 3)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 3)
    input_shape = (img_rows, img_cols, 3)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(np.ravel(y_train), num_classes)
y_test = keras.utils.to_categorical(np.ravel(y_test), num_classes)

Downloading data from https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz

 8192/170498071 [..............................] - ETA: 24:39
 40960/170498071 [..............................] - ETA: 9:54 
 106496/170498071 [..............................] - ETA: 5:42
 221184/170498071 [..............................] - ETA: 3:39
 450560/170498071 [..............................] - ETA: 2:14
 892928/170498071 [..............................] - ETA: 1:21
 1794048/170498071 [..............................] - ETA: 46s 
 3596288/170498071 [..............................] - ETA: 26s
 6725632/170498071 [>.............................] - ETA: 15s
 9773056/170498071 [>.............................] - ETA: 11s
 12886016/170498071 [=>............................] - ETA: 9s 
 15638528/170498071 [=>............................] - ETA: 8s
 18685952/170498071 [==>...........................] - ETA: 7s
 21708800/170498071 [==>...........................] - ETA: 6s
 23388160/170498071 [===>..........................] - ETA: 6s
 25100288/170498071 [===>..........................] - ETA: 6s
 27516928/170498071 [===>..........................] - ETA: 5s
 30466048/170498071 [====>.........................] - ETA: 5s
 33497088/170498071 [====>.........................] - ETA: 5s
 36315136/170498071 [=====>........................] - ETA: 4s
 38035456/170498071 [=====>........................] - ETA: 4s
 39624704/170498071 [=====>........................] - ETA: 4s
 42098688/170498071 [======>.......................] - ETA: 4s
 44900352/170498071 [======>.......................] - ETA: 4s
 47816704/170498071 [=======>......................] - ETA: 4s
 50929664/170498071 [=======>......................] - ETA: 4s
 53780480/170498071 [========>.....................] - ETA: 3s
 56713216/170498071 [========>.....................] - ETA: 3s
 59654144/170498071 [=========>....................] - ETA: 3s
 61399040/170498071 [=========>....................] - ETA: 3s
 63135744/170498071 [==========>...................] - ETA: 3s
 65347584/170498071 [==========>...................] - ETA: 3s
 68116480/170498071 [==========>...................] - ETA: 3s
 70721536/170498071 [===========>..................] - ETA: 3s
 73719808/170498071 [===========>..................] - ETA: 3s
 76554240/170498071 [============>.................] - ETA: 2s
 79650816/170498071 [=============>................] - ETA: 2s
 82714624/170498071 [=============>................] - ETA: 2s
 85368832/170498071 [==============>...............] - ETA: 2s
 87990272/170498071 [==============>...............] - ETA: 2s
 91086848/170498071 [===============>..............] - ETA: 2s
 94134272/170498071 [===============>..............] - ETA: 2s
 97001472/170498071 [================>.............] - ETA: 2s
 99983360/170498071 [================>.............] - ETA: 2s
102981632/170498071 [=================>............] - ETA: 1s
105717760/170498071 [=================>............] - ETA: 1s
108486656/170498071 [==================>...........] - ETA: 1s
111386624/170498071 [==================>...........] - ETA: 1s
113475584/170498071 [==================>...........] - ETA: 1s
115720192/170498071 [===================>..........] - ETA: 1s
116629504/170498071 [===================>..........] - ETA: 1s
119463936/170498071 [====================>.........] - ETA: 1s
121856000/170498071 [====================>.........] - ETA: 1s
123895808/170498071 [====================>.........] - ETA: 1s
125140992/170498071 [=====================>........] - ETA: 1s
127623168/170498071 [=====================>........] - ETA: 1s
130752512/170498071 [======================>.......] - ETA: 1s
133685248/170498071 [======================>.......] - ETA: 1s
136585216/170498071 [=======================>......] - ETA: 0s
139386880/170498071 [=======================>......] - ETA: 0s
142434304/170498071 [========================>.....] - ETA: 0s
144818176/170498071 [========================>.....] - ETA: 0s
146694144/1704

# Define function to construct CNN model

In [6]:
def cnn_model() : 
    model = Sequential()
    model.add(Conv2D(4, kernel_size=(3, 3),activation='relu',input_shape=input_shape))
    model.add(Conv2D(8, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(16, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    return(model)

# Define function to plot model accuracy

In [8]:
def plot_fig(i, history):
    fig = plt.figure()
    plt.plot(range(1,epochs+1),history.history['val_accuracy'],label='validation')
    plt.plot(range(1,epochs+1),history.history['accuracy'],label='training')
    plt.legend(loc=0)
    plt.xlabel('epochs')
    plt.ylabel('accuracy')
    plt.xlim([1,epochs])
    plt.grid(True)
    plt.title("Model Accuracy")
    # If you're running this notebook on databricks, use display to generate graphs
    display(fig)
    # If you're running this notebook locally, use plt.show to generate graphs
    # plt.show()
    # plt.close(fig)
    

# Constant learning rate

In [10]:
# define CNN model
model1 = cnn_model()

# define SGD optimizer
learning_rate = 0.1
sgd = SGD(lr=learning_rate, momentum=0.0, decay=0.0, nesterov=False) 

# compile the model
model1.compile(loss=keras.losses.categorical_crossentropy, optimizer=sgd, metrics=['accuracy'])

# fit the model
history1 = model1.fit(X_train, y_train,
                batch_size=batch_size,
                epochs=epochs,
                verbose=2,
                validation_data=(X_test, y_test))

# plot model accuracy
plot_fig(1, history1)

# Time-based decay

In [12]:
learning_rate = 0.1
decay_rate = learning_rate / epochs
lrs = [learning_rate] * epochs
for i in range(1, epochs):
    lrs[i] = lrs[i - 1] * (1. / (1. + decay_rate * i))

# plot learning rate
fig = plt.figure()
plt.plot(range(1,epochs+1),lrs,label='learning rate')
plt.xlabel("epoch")
plt.xlim([1,epochs+1])
plt.ylabel("learning rate")
plt.legend(loc=0)
plt.grid(True)
plt.title("Learning rate")
# If you're running this notebook on databricks, use display to generate graphs
display(fig)
# If you're running this notebook locally, use plt.show to generate graphs
# plt.show()
# plt.close(fig)

In [13]:
# define CNN model
model2 = cnn_model()

# define SGD optimizer
learning_rate = 0.1
decay_rate = 0.002 # learning_rate / epochs
momentum = 0.5
sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)

# compile the model
model2.compile(loss=keras.losses.categorical_crossentropy, optimizer=sgd, metrics=['accuracy'])

# fit the model
history2 = model2.fit(X_train, y_train, 
                     epochs=epochs, 
                     batch_size=batch_size,
                     verbose=2, 
                     validation_data=(X_test, y_test))

# plot model accuracy
plot_fig(2, history2)

# Step decay

In [15]:
learning_rate = 0.1
drop = 0.5
epochs_drop = 10.0
lrs = [learning_rate] * epochs
for i in range(1, epochs):
    lrs[i] = learning_rate * math.pow(drop, math.floor((i)/epochs_drop))
  
# plot learning rate
fig = plt.figure()
plt.plot(range(1,epochs+1),lrs,label='learning rate')
plt.xlabel("epoch")
plt.xlim([1,epochs+1])
plt.ylabel("learning rate")
plt.legend(loc=0)
plt.grid(True)
plt.title("Learning rate")
# If you're running this notebook on databricks, use display to generate graphs
display(fig)
# If you're running this notebook locally, use plt.show to generate graphs
# plt.show()
# plt.close(fig)

In [16]:
# define CNN model
model3 = cnn_model()

# define SGD optimizer
momentum = 0.5
sgd = SGD(lr=0.0, momentum=momentum, decay=0.0, nesterov=False) 

# compile the model
model3.compile(loss=keras.losses.categorical_crossentropy,optimizer=sgd, metrics=['accuracy'])

# define step decay function
def step_decay(epoch):
    initial_lrate = 0.1
    drop = 0.5
    epochs_drop = 10.0
    lrate = initial_lrate * math.pow(drop, math.floor((epoch)/epochs_drop))
    return lrate

# learning schedule callback
lrate = LearningRateScheduler(step_decay)
callbacks_list = [lrate]

# fit the model
history3 = model3.fit(X_train, y_train, 
                     validation_data=(X_test, y_test), 
                     epochs=epochs, 
                     batch_size=batch_size, 
                     callbacks=callbacks_list, 
                     verbose=2)

# plot model accuracy
plot_fig(3, history3)

# Exponential decay

In [18]:
learning_rate = 0.1
k = 0.1
lrs = [learning_rate] * epochs
for i in range(1, epochs):
    lrs[i] = learning_rate * np.exp(-k*i)

# plot learning rate
fig = plt.figure()
plt.plot(range(1,epochs+1),lrs,label='learning rate')
plt.xlabel("epoch")
plt.xlim([1,epochs+1])
plt.ylabel("learning rate")
plt.legend(loc=0)
plt.grid(True)
plt.title("Learning rate")
# If you're running this notebook on databricks, use display to generate graphs
display(fig)
# If you're running this notebook locally, use plt.show to generate graphs
# plt.show()
# plt.close(fig)

In [19]:
# define CNN model
model4 = cnn_model()

# define SGD optimizer
momentum = 0.5
sgd = SGD(lr=0.0, momentum=momentum, decay=0.0, nesterov=False)

# compile the model
model4.compile(loss=keras.losses.categorical_crossentropy, optimizer=sgd, metrics=['accuracy'])

# define step decay function
def exp_decay(epoch):
    initial_lrate = 0.1
    k = 0.1
    lrate = initial_lrate * np.exp(-k*epoch)
    return lrate

# learning schedule callback
lrate_ = LearningRateScheduler(exp_decay)
callbacks_list_ = [lrate_]

# fit the model
history4 = model4.fit(X_train, y_train, 
     validation_data=(X_test, y_test), 
     epochs=epochs, 
     batch_size=batch_size, 
     callbacks=callbacks_list_, 
     verbose=2)

# plot model accuracy
plot_fig(4, history4)